In [ ]:
!uv pip install -U lighteval accelerate
!uv pip install -U lighteval[multilingual]
!uv pip install lighteval[extended_tasks]

[benchmark index](https://huggingface.co/spaces/SaylorTwift/open_benchmark_index)

In [ ]:
from transformers import AutoModelForCausalLM

from lighteval.logging.evaluation_tracker import EvaluationTracker
from lighteval.models.transformers.transformers_model import TransformersModel, TransformersModelConfig
from lighteval.pipeline import ParallelismManager, Pipeline, PipelineParameters


MODEL_NAME = "Qwen/Qwen3-0.6B"
# format: {suite}|{task}|{num_few_shot}
# run multiple: with commas, e.g., "leaderboard|truthfulqa:mc|0,leaderboard|gsm8k|3"
BENCHMARKS = "leaderboard|truthfulqa:mc|0"

evaluation_tracker = EvaluationTracker(output_dir=f"./tmp/results")
pipeline_params = PipelineParameters(
    launcher_type=ParallelismManager.NONE,
    max_samples=2
)

model = AutoModelForCausalLM.from_pretrained(
  MODEL_NAME, device_map="auto"
)
config = TransformersModelConfig(model_name=MODEL_NAME, batch_size=1)
model = TransformersModel.from_model(model, config)

pipeline = Pipeline(
    model=model,
    pipeline_parameters=pipeline_params,
    evaluation_tracker=evaluation_tracker,
    tasks=BENCHMARKS,
)

results = pipeline.evaluate()
pipeline.show_results()
results = pipeline.get_results()

In [ ]:
#!lighteval tasks list
#!lighteval tasks list --suites lighteval

#!lighteval accelerate "model_name=Qwen/Qwen3-0.6B" "leaderboard|truthfulqa:mc|0" --max-samples 2 --output-dir ./tmp/results